In [1]:
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import re
import string

sns.set(); 

In [2]:
data = pd.read_csv('rabotaru_ru/data/train_public.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56297 entries, 0 to 56296
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           56297 non-null  int64  
 1   custom_position              56297 non-null  object 
 2   operating_schedule_id        56297 non-null  int64  
 3   description                  56297 non-null  object 
 4   salary_from                  56297 non-null  int64  
 5   offer_education_id           56297 non-null  int64  
 6   age_from                     15798 non-null  float64
 7   age_to                       12931 non-null  float64
 8   is_agency                    56297 non-null  bool   
 9   is_nonresident               56297 non-null  bool   
 10  offer_experience_year_count  38622 non-null  float64
 11  salary_to                    34829 non-null  float64
 12  is_male                      8804 non-null   object 
 13  driving_license 

In [4]:
def change_columns_type(data):
    # меняем тип некоторых колонок на категориальный
    data['operating_schedule_id'] = data['operating_schedule_id'].astype('object')
    data['is_male'] = data['is_male'].astype('object')
    data['company_id'] = data['company_id'].astype('object')
    data['is_agency'] = data['is_agency'].astype('object')
    data['is_nonresident'] = data['is_nonresident'].astype('object')
    data['offer_education_id'] = data['offer_education_id'].astype('object')
    data['city_id'] = data['city_id'].astype('object')

In [5]:
change_columns_type(data)

In [6]:
pd.set_option('display.max_rows', 1500)

In [7]:
data['region_id'].nunique()

1486

In [8]:
data['region_id'].unique()

array(['{1783}', '{3}', '{5807}', ..., '{3419}', '{958}', '{3059}'],
      dtype=object)

In [9]:
# удаляем фигурные скобки из региона
data['region_id'].replace('[{}]','',regex=True, inplace=True)

In [10]:
data['region_id'].describe()

count     56297
unique     1486
top           3
freq      18167
Name: region_id, dtype: object

In [11]:
rvc = data['region_id'].value_counts(normalize = False)
rvc.to_dict().values()

dict_values([18167, 6654, 1515, 1162, 965, 932, 745, 716, 716, 551, 550, 497, 485, 406, 391, 378, 374, 361, 353, 348, 337, 324, 310, 304, 273, 268, 265, 264, 263, 256, 255, 255, 253, 247, 244, 244, 220, 217, 216, 202, 197, 196, 196, 192, 186, 171, 164, 157, 153, 150, 148, 142, 138, 135, 129, 126, 126, 125, 121, 118, 117, 117, 114, 113, 110, 109, 109, 105, 105, 104, 104, 104, 100, 97, 96, 94, 93, 89, 89, 87, 84, 81, 81, 80, 79, 79, 79, 79, 78, 78, 77, 77, 76, 75, 73, 72, 70, 69, 66, 66, 63, 62, 62, 61, 61, 60, 60, 60, 59, 58, 58, 58, 57, 55, 53, 53, 51, 51, 50, 49, 49, 49, 48, 48, 47, 47, 46, 46, 46, 46, 45, 45, 45, 45, 44, 43, 43, 42, 41, 41, 41, 41, 41, 40, 40, 39, 39, 38, 38, 38, 38, 37, 37, 36, 36, 36, 35, 35, 35, 35, 35, 35, 34, 34, 34, 34, 33, 33, 32, 32, 32, 32, 31, 31, 31, 30, 30, 30, 30, 29, 29, 28, 27, 27, 26, 26, 26, 25, 25, 25, 25, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 18, 17, 17, 17, 17, 17, 16, 16, 1

In [12]:
rvc = data['region_id'].value_counts().loc[lambda x : x > 10]
regions_more_10 = list(rvc.keys())

In [13]:
# проверяем не встречается ли значение региона 0
data[ data['region_id'] == 0 ]

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof


In [14]:
#df.loc[df.my_channel > 20000, 'my_channel'] = 0
data.loc[ ~data['region_id'].isin(regions_more_10), 'region_id'] = 0

In [15]:
# строки, регионы которых встречаются реже 10 раз
data[ ~data['region_id'].isin(regions_more_10) ]

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof
3,365230,Разнорабочий,7,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,35000,0,NaN,NaN,False,True,-100.0,40000.0,NaN,[],0,2.78416e+06,70,разнорабочий
44,413863,Повар-универсал,7,"Вакансия компании: ООО ""ПитРесурс-Таврида<br /...",50000,2,NaN,NaN,True,True,1.0,NaN,NaN,[],0,1.00121e+06,5,повар
47,288888,Продавец-кассир (г. Почеп),3,"<strong>Всем новым сотрудникам, оформившимся в...",23863,0,NaN,NaN,False,False,-100.0,NaN,NaN,[],0,776587,255,кассир
73,360130,Продавец-консультант,1,Знание 1С<br />\r\nКонсультирование покупателе...,40000,0,NaN,NaN,False,True,1.0,60000.0,NaN,[],0,2.78136e+06,82,консультант
86,239537,Специалист по прямым продажам,1,Если клиент получает зарплату на карту Сбербан...,39000,2,NaN,NaN,False,True,-1.0,NaN,NaN,[],0,788450,16,специалист
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56248,410310,Курьер в Яндекс Доставка,2,<strong>Вакансия компании: Яндекс Доставка</st...,-1,0,NaN,NaN,True,False,-100.0,-1.0,NaN,[],0,1.00121e+06,8,курьер
56265,50541,Специалист по продаже и установке спутникового ТВ,5,<strong>МТС &ndash; крупнейший телеком-бренд в...,30000,0,NaN,NaN,False,False,NaN,NaN,NaN,[],0,1.05588e+06,27,специалист
56288,467960,Технолог пищевого производства (выпечка печенья),1,На пищевое производство (выпечка печенья) в г....,25000,0,NaN,NaN,False,False,NaN,30000.0,NaN,[],0,2.98603e+06,63,технолог
56290,195878,Работник торгового зала,3,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,24024,0,18.0,NaN,False,True,-100.0,NaN,NaN,[],0,1.7532e+06,6,работник


In [16]:
rvc[:10]

3      18167
4       6654
231     1515
240     1162
310      965
25       932
328      745
529      716
18       716
284      551
Name: region_id, dtype: int64

In [17]:
data['city_id'].nunique()

67

In [18]:
data['city_id'].unique()

array([70, 1, 258, 20, 256, 2, 3, 255, 38, 12, 50, 260, 7, 16, 57, 249,
       32, 15, 10, 9, 29, 6, 8, 35, 5, 36, 4, 19, 64, 49, 30, 86, 82, 28,
       11, 83, 102, 14, 27, 63, 24, 251, 37, 13, 269, 254, 51, 26, 17,
       261, 18, 268, 259, 272, 174, 54, 270, 22, 89, 247, 34, 250, 99,
       252, 148, 263, 111], dtype=object)

In [19]:
data['city_id'].describe()

count     56297
unique       67
top           1
freq      16144
Name: city_id, dtype: int64

In [20]:
data['city_id'].value_counts(normalize = False)

1      16144
2       5771
57      4509
4       1845
16      1338
9       1323
15      1232
102     1034
14       977
3        895
12       828
6        808
8        765
35       760
17       757
5        739
19       738
20       671
13       612
49       573
7        554
28       542
27       499
50       497
36       494
18       493
82       491
11       489
10       485
63       469
29       447
64       438
32       408
24       397
34       394
70       387
22       366
174      358
30       350
255      345
269      329
38       315
37       292
26       287
259      276
258      265
83       251
254      242
256      229
261      220
247      218
250      217
86       214
272      210
51       210
249      203
251      189
268      182
99       151
89       148
54       145
270      126
148       81
260       56
252       16
263        2
111        1
Name: city_id, dtype: int64

In [21]:
data['company_id'].nunique()

10218

In [22]:
cvc = data['company_id'].value_counts().loc[lambda x : x > 10]
#regions_more_10 = list(rvc.keys())
len(cvc)

484

In [23]:
data['target_prof'].unique()

array(['рабочий', 'разнорабочий', 'повар', 'представитель', 'слесарь',
       'специалист', 'электромонтер', 'колорист', 'водитель',
       'администратор', 'оператор', 'резчик', 'плотник', 'вальцовщик',
       'горничная', 'охранник', 'курьер', 'овощевод', 'кассир',
       'hr-менеджер', 'уборщик', 'продавец', 'бухгалтер', 'механизатор',
       'менеджер', 'швея', 'строитель', 'упаковщик', 'кладовщик',
       'раскладчик', 'монтажник', 'официант', 'каменщик', 'трейдер',
       'репетитор', 'комплектовщик', 'приемщик', 'электромонтажник',
       'маляр', 'фрезеровщик', 'консультант', 'логист', 'сварщик',
       'заместитель', 'машинист', 'санитар', 'врач', 'мойщик', 'инженер',
       'пекарь', 'котломойщик', 'сантехник', 'сортировщик', 'инспектор',
       'автомаляр', 'начальник', 'электромеханик', 'проводник', 'грузчик',
       'дворник', 'электрик', 'боец', 'тракторист', 'бетонщик', 'сборщик',
       'укладчик', 'заправщик', 'мерчендайзер', 'гардеробщик', 'лаборант',
       'тренер',

In [24]:
for prof in data['target_prof'].unique():
    print(prof)

рабочий
разнорабочий
повар
представитель
слесарь
специалист
электромонтер
колорист
водитель
администратор
оператор
резчик
плотник
вальцовщик
горничная
охранник
курьер
овощевод
кассир
hr-менеджер
уборщик
продавец
бухгалтер
механизатор
менеджер
швея
строитель
упаковщик
кладовщик
раскладчик
монтажник
официант
каменщик
трейдер
репетитор
комплектовщик
приемщик
электромонтажник
маляр
фрезеровщик
консультант
логист
сварщик
заместитель
машинист
санитар
врач
мойщик
инженер
пекарь
котломойщик
сантехник
сортировщик
инспектор
автомаляр
начальник
электромеханик
проводник
грузчик
дворник
электрик
боец
тракторист
бетонщик
сборщик
укладчик
заправщик
мерчендайзер
гардеробщик
лаборант
тренер
товаровед
мастер
электросварщик
военнослужащий
фасовщик
штукатур
формовщик
токарь
дизайнер
замерщик
прораб
сотрудник
раздатчик
полировщик
координатор
контролер
диспетчер
электрогазосварщик
работник
монтер
стропальщик
домработник
шлифовщик
кузнец
автоэлектрик
техник
экономист
риэлтор
гипсокартонщик
сторож
developer
и

In [25]:
data[ data['target_prof'] == 'тестировщик' ]

,id,custom_position,operating_schedule_id,description,salary_from,offer_education_id,age_from,age_to,is_agency,is_nonresident,offer_experience_year_count,salary_to,is_male,driving_license,region_id,company_id,city_id,target_prof
10080,29142,QA Java Automation (База Знаний),1,"<p>Наш продукт &ndash; это система, которая яв...",-1,4,NaN,NaN,False,True,-1.0,-1.0,NaN,[],3,788450,1,тестировщик
20081,312192,Тестировщик ПО,6,Работа.ру &mdash; один из лидеров на рынке тру...,-1,0,NaN,NaN,False,True,1.0,-1.0,NaN,[],4,1.73452e+06,1,тестировщик
21722,245961,QA Automation Middle (удаленно),1,<strong>Обязанности:</strong><br /> &bull; Нап...,100000,0,NaN,NaN,False,False,NaN,120000.0,NaN,[],3,1.25397e+06,1,тестировщик
24071,308670,Тестировщик,1,<strong>На КРУПНЕЙШЕЕ производство электронног...,39000,0,NaN,NaN,False,True,-1.0,NaN,NaN,[],4,1.78322e+06,2,тестировщик
25385,313962,Тестировщик,1,<strong>Вакансия компании</strong>: ООО Монито...,45000,2,NaN,NaN,False,True,3.0,NaN,NaN,[],3,1.7388e+06,1,тестировщик
41754,452592,Тестировщик,1,Центральный модуль управления операционным дне...,-1,4,NaN,NaN,False,True,1.0,-1.0,NaN,[],4,788450,2,тестировщик
42365,362086,Тестировщик программного обеспечения,1,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,-1,4,NaN,NaN,False,True,1.0,-1.0,NaN,[],3,1.74123e+06,1,тестировщик
48418,168443,Тестировщик (функциональное тестирование),1,<p><strong>Проект:</strong> Создание АС ЦАРС</...,-1,4,NaN,NaN,False,True,-1.0,-1.0,NaN,[],4,788450,2,тестировщик
55593,245962,QA Automation Senior (удаленно),1,<strong>Обязанности:</strong><br /> &bull; Вне...,130000,0,NaN,NaN,False,False,NaN,200000.0,NaN,[],3,1.25397e+06,1,тестировщик


In [26]:
pvc = data['target_prof'].value_counts(normalize = False)
pvc

водитель                4885
продавец                3360
менеджер                2399
повар                   2284
грузчик                 2137
курьер                  2034
уборщик                 1841
охранник                1604
кассир                  1542
комплектовщик           1528
оператор                1425
упаковщик               1252
специалист              1176
слесарь                 1121
разнорабочий            1070
рабочий                  904
сварщик                  877
инженер                  832
работник                 812
монтажник                778
машинист                 752
токарь                   724
сборщик                  712
мастер                   690
врач                     615
кладовщик                576
консультант              532
администратор            521
начальник                520
бухгалтер                422
фасовщик                 395
официант                 389
дворник                  372
электромонтажник         346
швея          

In [33]:
tpvc_less_5 = data['target_prof'].value_counts().loc[lambda x : x <= 2]
len(tpvc_less_5)

166

In [34]:
tpvc_less_5 = data['target_prof'].value_counts().loc[lambda x : x <= 2]
prof_less_5 = list(tpvc_less_5.keys())
prof_less_5

['контроллер',
 'скрутчик',
 'наполнитель',
 'директолог',
 'тандырщик',
 'сдатчик',
 'обжигальщик',
 'маркшейдер',
 'раздельщик',
 'data scientist',
 'интервьюер',
 'детектив',
 'эвакуатор',
 'тигельщик',
 'гидрогеолог',
 'манипуляторщик',
 'коптильщик',
 'обойщик',
 'участковый',
 'ручник',
 'кальянщик',
 'детейлер',
 'кондуктор',
 'вышивальщик',
 'партнер',
 'альпинист',
 'метролог',
 'кадровик',
 'грохотовщик',
 'гидравлик',
 'холодильщик',
 'ткач',
 'опрессовщик',
 'копирайтер',
 'строгальщик',
 'обрезчик',
 'спасатель',
 'режиссер',
 'хореограф',
 'филетировщик',
 'скульптор',
 'оптометрист',
 'закупщик',
 'истопник',
 'чертежник',
 'тепличник',
 'волочильщик',
 'кромщик',
 'расфасовщик',
 'гравировщик',
 'техслужащий',
 'гидроизолировщик',
 'банщик',
 'горнорабочий',
 'куратор',
 'варщик',
 'трейдер',
 'командир',
 'вентиляционщик',
 'адвокат',
 'приборист',
 'радиомонтажник',
 'животновод',
 'барбер',
 'видеооператор',
 'паркетчик',
 'паспортист',
 'колеровщик',
 'табельщик',
 

In [ ]:
data.drop( data['target_prof'].isin(prof_less_5).index, inplace=False )  